In [2]:
%load_ext autoreload
import sys
sys.path.append("../..")

# Helium $f(z)$ Calculation

In [14]:
%autoreload


import matplotlib.pyplot as plt
%matplotlib inline


import matplotlib
matplotlib.rc_file('matplotlibrc')
matplotlib.rcParams['figure.figsize'] = [10,10]


# SET THIS TO TRUE TO USE LATEX
matplotlib.rcParams['text.usetex'] = True

from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.low_energy.lowE_deposition import compute_fs
import main

from tqdm import tqdm_notebook as tqdm

from astropy.io import fits


# np.set_printoptions(threshold=np.nan)

## Run DarkHistory

In [16]:
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190120/'

highengphot_tf_interp = pickle.load(open(direc+"/processed/highengphot_tf_interp.raw", "rb"))
lowengphot_tf_interp  = pickle.load(open(direc+"/processed/lowengphot_tf_interp.raw", "rb"))
lowengelec_tf_interp  = pickle.load(open(direc+"/processed/lowengelec_tf_interp.raw", "rb"))
highengdep_interp     = pickle.load(open(direc+"/processed/highengdep_interp.raw", "rb"))
CMB_engloss_interp    = pickle.load(open(direc+"/processed/CMB_engloss_interp.raw", "rb"))

In [22]:
direc = "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/"
ics_thomson_ref_tf = pickle.load(open(direc+"ics_thomson_ref_tf.raw", "rb"))
ics_rel_ref_tf = pickle.load(open(direc+"ics_rel_ref_tf.raw", "rb"))
engloss_ref_tf = pickle.load(open(direc+"engloss_ref_tf.raw", "rb"))

In [25]:
#Annihilation or decay?
inj_type = 'decay'

#Annihilate/Decay into
inj_particle = 'photon'

#Useful definitions
if inj_particle == 'electron':
    inj_str = 'elec'
    mass = phys.me
else:
    inj_str = 'phot'
    mass = 0
    
CMB_subtracted = True

In [48]:
%autoreload

#injection energy, DM mass, center-of-mass energy
ind = 2

log10eng0 = 3.6989700794219966
log10eng = np.array([log10eng0 + 0.23252559*i for i in np.arange(39)])

tau=.5e25
sigmav=0
end_rs = 2990
start_rs = 3000

photeng = highengphot_tf_interp.eng
eleceng = lowengelec_tf_interp.eng

Einj = 10**log10eng[ind]
mDM = 2*Einj + 2*mass
print(np.log10(mDM))
Ecom=mDM

def rate_func_N(rs):
    return phys.inj_rate(inj_type, rs, mDM=mDM, sigmav=sigmav, tau=tau)/Ecom
def rate_func_eng(rs):
    return phys.inj_rate(inj_type, rs, mDM=mDM, sigmav=sigmav, tau=tau)

if inj_type == 'swave':
    mDM = mDM/2
    Ecom = 2*mDM   

if inj_particle == 'electron':
    Etmp = eleceng[eleceng < Einj][-1]
    elec_spec_init = spectools.rebin_N_arr(
        np.array([2*Einj/Etmp]),
        np.array([Etmp]),
        eleceng
    )
    elec_spec_init.rs = start_rs
    elec_spec_init.switch_spec_type()

    phot_spec_init = Spectrum(photeng,
                             np.zeros_like(photeng),
                             spec_type='N',
                             rs=start_rs)
else:
    Etmp = photeng[photeng < Einj][-1]
    phot_spec_init = spectools.rebin_N_arr(
        np.array([2*Einj/Etmp]),
        np.array([Etmp]),
        photeng
    )
    phot_spec_init.rs = start_rs
    phot_spec_init.switch_spec_type()

    elec_spec_init = Spectrum(eleceng,
                             np.zeros_like(eleceng),
                             spec_type='N',
                             rs=start_rs)

f_DH_list = main.evolve(
    elec_spec_init, phot_spec_init,
    rate_func_N, rate_func_eng, end_rs,
    highengphot_tf_interp, lowengphot_tf_interp, lowengelec_tf_interp,
    highengdep_interp, CMB_engloss_interp,
    ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
    coarsen_factor=1, std_soln=True,
    reion_switch=False,
    struct_boost=None,
    ics_only=False, separate_higheng=True, CMB_subtracted=CMB_subtracted,
    user=None, verbose=False, 
    use_tqdm=True, compute_fs_method='helium'
)

4.465051255085978


huh???
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1.